# Testing program 
## Non linear Joints Bachelor project 2019-2020
### Maxime Fellrath, 274858

In [1]:
import ipywidgets as widgets
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt
import random
import igl
import math
import cv2 
import copy
import networkx as nx
import shapely.geometry as shy
from PIL import Image, ImageOps 
from shapely.ops import cascaded_union
from matplotlib.pyplot import figure, draw, pause
from ipywidgets import interact, interactive, fixed, interact_manual, FloatSlider
from PIL import Image, ImageDraw
from matplotlib.patches import Circle, Wedge, Polygon
from matplotlib.collections import PatchCollection
from matplotlib.pyplot import figure
from numpy import linalg as LA
epsilon=1e-5
from shapely.geometry import Point, MultiPoint
colors= np.array(['#85C17E','#E2BC74','#425B8A','#B666D2','#9393b3','#85C17E','#E2BC74','#425B8A','#B666D2','#85C17E' ,'#85C17E','#E2BC74','#425B8A','#B666D2','#85C17E' ,'#85C17E','#E2BC74','#425B8A','#B666D2','#85C17E' ,'#85C17E','#E2BC74','#425B8A','#B666D2','#85C17E' ,'#85C17E','#E2BC74','#425B8A','#B666D2','#85C17E' ,'#85C17E','#E2BC74','#425B8A','#B666D2','#85C17E' ,'#85C17E','#E2BC74','#425B8A','#B666D2','#85C17E' ,'#85C17E','#E2BC74','#425B8A','#B666D2','#85C17E' ,'#85C17E','#E2BC74','#425B8A','#B666D2','#85C17E'   ])
colors = np.array(['#FFF8DC', '#FFEBCD','#FFE4C4','#FFDEAD','#F5DEB3','#DEB887','#D2B48C','#BC8F8F','#F4A460','#DAA520','#B8860B','#CD853F','#D2691E','#8B4513','#A52A2A','#800000'])
#colors = np.array(['#e6194b', '#3cb44b', '#ffe119', '#4363d8', '#f58231', '#911eb4', '#46f0f0', '#f032e6', '#bcf60c', '#fabebe', '#008080', '#e6beff', '#9a6324', '#fffac8', '#800000', '#aaffc3', '#808000', '#ffd8b1', '#000075', '#808080', '#ffffff', '#000000'])
#colors = np.array(['#FFE4C4','#BC8F8F','#CD853F','#D2691E','#8B4513','#A52A2A','#CD853F','#CD853F','#CD853F','#CD853F','#CD853F','#CD853F','#CD853F','#CD853F','#CD853F','#CD853F','#CD853F','#CD853F','#CD853F','#CD853F','#CD853F','#CD853F','#CD853F'])



# Image operation functions

In [2]:
# code inspiration : https://stackoverflow.com/questions/47528222/how-to-get-only-the-cropped-portion-of-an-image-in-python
def createImages(piece1, img, l= 2800):
    for i in range(16):
        im = Image.open("ressources/"+str(img)).convert("RGBA")
        im = im.transpose(Image.FLIP_TOP_BOTTOM)

        im = im.resize((l, l), Image.ANTIALIAS)
        imArray = np.asarray(im)

        maskIm = Image.new('L', (imArray.shape[1], imArray.shape[0]), 0)
        tmp = [(piece1[i][j][0]*(l)/4000,piece1[i][j][1]*(l)/4000) for j in range(len(piece1[i])) ]

        ImageDraw.Draw(maskIm).polygon(tmp, outline=0, fill=1)
        mask = np.array(maskIm)

        newImArray = np.empty(imArray.shape,dtype='uint8')
        newImArray[:,:,:3] = imArray[:,:,:3]
        newImArray[:,:,3] = mask*255
        newIm = Image.fromarray(newImArray, "RGBA")


        newIm.save("images/"+str(img)+"out"+str(i)+".png")
        

In [3]:
def translateImg(img, tx, ty):
 
    im2 = ImageOps.expand(img, border = 20, fill = 50)
    im2 = im2.transform(im2.size, Image.AFFINE, (1, 0, tx, 0, 1, ty))
    return im2

# Display Function

In [11]:

def displayPolygonsStable(figure, v, k=0, image = None, l= 30): 
    clone = copy.deepcopy(figure)
    def update(m=0, size= l):
        patches = []
        xmax=size+200
        ymax=size+200
        xmin = -200-size/200
        ymin=-200-size/200
        k=0
        fig2 = plt.figure(num=None, figsize=(8, 8))
        ax2 = fig2.gca()
        if(v.size>0):
            for k in range(len(figure)):
                for l in range(len(figure[k])):
                    clone[k][l][0]=figure[k][l][0] + m * v[2*k] 
                    clone[k][l][1]=figure[k][l][1] + m * v[2*k+1]

        for i in range(len(figure)):
            
            if(image == None):
                polygon = Polygon(clone[i], fill=True, facecolor=colors[i],edgecolor='black',label='figure'+str(i))
                ax2.add_artist(polygon)
            else:
                newIm = Image.open("images/"+str(image)+"out"+str(i)+".png")
                newIm = translateImg(newIm, -m * v[2*i] ,-m * v[2*i+1] )

                h = ax2.imshow(newIm)  

        ax2.set(xlim=(xmin, xmax), ylim=(ymin, ymax))
            

    interact(update,m=FloatSlider(min=0, max=100, step=1))
    

def displayPolygons(figure, v, k=0): 
    clone = copy.deepcopy(figure)
    def update(i=0):
        patches = []
        xmax=0
        ymax=0
        xmin =0
        ymin=0
        k=0
        fig2 = plt.figure(num=None, figsize=(8, 8))
        ax2 = fig2.gca()
        if(v.size>0):
            for k in range(len(figure)):
                for l in range(len(figure[k])):
                    clone[k][l][0]=figure[k][l][0] + i * v[2*k] 
                    clone[k][l][1]=figure[k][l][1] + i * v[2*k+1]

        for i in range(len(figure)):
            polygon = Polygon(clone[i], fill=True, facecolor=colors[i],edgecolor='black',label='figure'+str(i))
            ax2.add_artist(polygon)
            if(xmax<np.max(clone[i][:,0])):
                xmax=np.max(clone[i][:,0])
            if(ymax<np.max(clone[i][:,1])):
                ymax=np.max(clone[i][:,1])
            if(xmin>np.min(clone[i][:,0])):
                xmin=np.min(clone[i][:,0])
            if(ymin>np.min(clone[i][:,1])):
                ymin=np.min(clone[i][:,1])
        ax2.set(xlim=(xmin, xmax), ylim=(ymin, ymax))
            

    interact(update,i=FloatSlider(min=0, max=10, step=0.1))
def simpleDisplay(figure):
    patches = []
    xmax=0
    ymax=0
    xmin =0
    ymin=0
    k=0
    fig2 = plt.figure(num=None, figsize=(8, 8))
    ax2 = fig2.gca()
    for i in range(len(figure)):
        polygon = Polygon(figure[i], fill=True, facecolor=colors[i],edgecolor='black',label='figure'+str(i))
        ax2.add_artist(polygon)
        if(xmax<np.max(figure[i][:,0])):
            xmax=np.max(figure[i][:,0])
        if(ymax<np.max(figure[i][:,1])):
            ymax=np.max(figure[i][:,1])
        if(xmin>np.min(figure[i][:,0])):
            xmin=np.min(figure[i][:,0])
        if(ymin>np.min(figure[i][:,1])):
            ymin=np.min(figure[i][:,1])
        ax2.set(xlim=(xmin, xmax), ylim=(ymin, ymax))
    plt.show()

# normals computing functions

In [5]:
def cross(a,b):
    return a[0]*b[1]-b[0]*a[1]

def is_between(a, b, c): # is point c between point a and b
    if(np.allclose(a,c) or np.allclose(b,c)):
        return False
    cross_product= cross((c-a),(b-a))
    if abs(cross_product) > epsilon:
        return False
    dotproduct = (b-a)@(c-a)
    if dotproduct < 0:
        return False
    squaredlengthba = np.sum(np.square(b-a))
    if dotproduct > squaredlengthba:
        return False
    return True

def isTouching(v1,v2, u1,u2):
    if((np.allclose(v1,u1) and np.allclose(v2,u2)) or  (np.allclose(v2,u1) and np.allclose(v1,u2))):
        return True
    return abs(cross(v1-v2, u1-u2)) < epsilon and (is_between(v1,v2,u1) or is_between(v1,v2,u2) or is_between(u1,u2,v1) or is_between(u1,u2,v2))  
def compute_normal_edge(v1,v2,i,j):
    normal = np.array(v2-v1)
    normal[0], normal[1] = normal[1], -normal[0]
    normal = normal / LA.norm(normal)
    return np.around(np.array(normal), decimals = 1)

def compute_normals_shape(shape):
    normals_of_shape= np.array([compute_normal_edge(shape[i],shape[(i+1)%len(shape)]) for i in range(len(shape))])
    return normals_of_shape

def compute_normals_2_shapes(shape1, shape2): 
    result = np.array([compute_normal_edge(shape1[i],shape1[(i+1)%len(shape1)],shape2[j],shape2[(j+1)%len(shape2)]) for i in range (len(shape1)) for j in range (len(shape2)) if(isTouching(shape1[i],shape1[(i+1)%len(shape1)],shape2[j],shape2[(j+1)%len(shape2)]))])   
    return set(tuple(i) for i in result)

def compute_normals_dict(figure):
    normals_dic = {}
    for i in range(len(figure)):
        for j in range(len(figure)):
            if(i != j):
                normals_dic[i,j]= compute_normals_2_shapes(figure[i], figure[j])
    return normals_dic

# inequation solver functions

In [6]:

def construct_A(figure):
    normals= compute_normals_dict(figure)
    count=0
    for x in normals: 
        count += len(normals[x]) 
    A = np.zeros((count,  len(figure)*2))
    c=0
    for x in normals:
        for j in range(len(normals[x])):
            a= list(normals[x])
            A[c][2*list(x)[0]]=-a[j][0]
            A[c][2*list(x)[0]+1]=-a[j][1]
            A[c][2*list(x)[1]]=a[j][0]
            A[c][2*list(x)[1]+1]=a[j][1]

            c=c+1
    return np.unique(A, axis=0)

def solver(A, i): #fixes all velocities except the i part to 0
    x = cp.Variable(len(A[0]))
    tij=cp.Variable(len(A))
    index=np.arange(len(A[0]))
    index= np.delete(index, 2*i)
    index= np.delete(index, 2*i)

    constraints = [(A*x-tij)>=np.zeros(len(A)), x[index]==0, tij>=0, tij<=1]

    objective =  cp.Maximize(cp.sum(tij))
    prob = cp.Problem(objective, constraints)
    result = prob.solve()
    return np.round(tij.value,decimals=3), np.round(x.value,decimals=3)

def solver_2(A, i,n, iskey=False,): #fixes only one reference part pr velocity to 0
    x = cp.Variable(len(A[0]))
    tij=cp.Variable(len(A))    
    if(iskey and n>2):
        constraints = [(A*x-tij)>=np.zeros(len(A)), x[(i+1)%len(A[0])]==0, x[i]==0, tij>=0, tij<=1]
    elif(n<=2):
        constraints = [(A*x-tij)>=np.zeros(len(A)),  tij>=0, tij<=1]
    else:
        constraints = [(A*x-tij)>=np.zeros(len(A)), x[(i+1)%len(A[0])]==0, tij>=0, tij<=1]

    objective =  cp.Maximize(cp.sum(tij))
    prob = cp.Problem(objective, constraints)
    result = prob.solve()
    return np.round(tij.value,decimals=3), np.round(x.value,decimals=3)

def solverMultipleMoving(A):
    x = cp.Variable(len(A[0]))
    tij=cp.Variable(len(A))    
    
    constraints = [(A*x-tij)>=np.zeros(len(A)), tij>=0, tij<=1]
    objective =  cp.Maximize(cp.sum(tij))
    prob = cp.Problem(objective, constraints)
    result = prob.solve()
    return np.round(tij.value,decimals=3), np.round(x.value,decimals=3)

# testing functions

In [7]:
def interlocking_test(figure, allVelocitiesFixed = True, key=None):
    A=construct_A(figure)
    final_v = np.array([])
    max_tij = np.zeros(len(A)) 
    
    for i in range(len(figure)):
        if(allVelocitiesFixed):
            temp_tij, v = solver(A, i)
        else:
            if(key is not None):
                temp_tij, v = solver_2(A, i,len(figure),True)
            else:
                temp_tij, v = solver_2(A, i,len(figure))

        if(np.sum(temp_tij) > np.sum(max_tij)):
            max_tij=temp_tij
            final_v=v
            
    return  max_tij, final_v   
def interlocking_test_moving(figure):
    A=construct_A(figure)
    final_v = np.array([])
    max_tij = np.zeros(len(A)) 
    temp_tij, v = solverMultipleMoving(A)
    if(np.sum(temp_tij) > np.sum(max_tij)):
        max_tij=temp_tij
        final_v=v

    return  max_tij, final_v   
def movingPieces(v):
    movingPieces=[]
    for i in range(len(v)//2):
        if(v[2*i]>0 or v[2*i+1]> 0):
            movingPieces.append(i)
    return movingPieces

#single piece moving function test and interactive plot
def fullStudy(figure, display = 0):
    if(display == 0):
        tij, v= interlocking_test(figure, True)
        print(v)
        if(v.size>0):
            if(len(movingPieces(v))>1):
                displayPolygons(figure, v)
                return "non interlocking, figure {} are able to move together".format(movingPieces(v))
            else:
                print("only piece "+str(movingPieces(v))+" can move, it's the key")
                displayPolygons(figure,v)
                if(v.size!=0):
                    return "it is an interlocking assembly ( deadlocking if key is fixed)"

        else:
            displayPolygons(figure,v)
            return "deadlocking"
    else:
        tij, v= interlocking_test(figure, True)
        print(v)
        if(v.size>0):
            if(len(movingPieces(v))>1):
                displayPolygonsStable(figure, v)
                return "non interlocking, figure {} are able to move together".format(movingPieces(v))
            else:
                print("only piece "+str(movingPieces(v))+" can move, it's the key")
                displayPolygonsStable(figure,v)
                tij, v = interlocking_test(figure, False, movingPieces(v)[0])
                if(v.size!=0):
                    return "it is an interlocking assembly ( deadlocking if key is fixed)"

        else:
            displayPolygonsStable(figure,v)
        
    return "deadlocking"

#Multiple Moving function test and interactive plot
def StudyMultipleMoving(figure, k=0, image = None, l=2800, recreate = False):
    if(image != None and recreate):
        createImages(figure, image, l)

    tij, v= interlocking_test_moving(figure)
    print(v)
    if(v.size>0):
            displayPolygonsStable(figure, v, k, image, l = 4000)
       
    else:
        displayPolygonsStable(figure,v, k, image, l=4000)
        return "deadlocking"
              

# Test with image: 

In [8]:
piece = np.load("numpyPieces/randomgrid.npy",allow_pickle = True)

In [9]:
StudyMultipleMoving(piece, image = "robert-bahn.jpg", l= 350, recreate= 0)

KeyboardInterrupt: 

Photo by Robert Bahn on Unsplash

# Array testing:

In [ ]:
#load piece to test, must have been saved using np.save("title")
to_test = np.load("numpyPieces/GridPieceArray.npy", allow_pickle = True)

In [ ]:
StudyMultipleMoving(to_test, l = 4000)

In [ ]:
to_test = np.load("numpyPieces/randomGrid.npy", allow_pickle = True)

In [ ]:
StudyMultipleMoving(to_test, image = "robert-bahn.jpg", l= 4000, recreate= 1)

In [14]:
tst = np.array([
np.array([
[ -14.282 , -4.891 ],
[ -15.698 , -3.476 ],
[ -18.633 , -6.194 ],
[ -17.218 , -7.609 ],
[ -22.5 , -12.5 ],
[ -15.0 , -25.0 ],
[ -5.0 , -25.0 ],
[ -1.952 , -16.873 ],
[ 0.184 , -16.873 ],
[ 1.588 , -13.127 ],
[ -0.548 , -13.127 ],
[ 2.5 , -5.0 ],
[ -1.416 , -3.297 ],
[ -1.416 , -1.117 ],
[ -5.084 , 0.478 ],
[ -5.084 , -1.703 ],
[ -9.0 , -0.0 ]
]),
np.array([
[ 18.24 , -12.926 ],
[ 18.24 , -14.936 ],
[ 14.26 , -15.334 ],
[ 14.26 , -13.324 ],
[ 10.0 , -13.75 ],
[ 2.5 , -5.0 ],
[ -0.548 , -13.127 ],
[ 1.588 , -13.127 ],
[ 0.184 , -16.873 ],
[ -1.952 , -16.873 ],
[ -5.0 , -25.0 ],
[ 15.0 , -25.0 ],
[ 22.5 , -12.5 ]
]),
np.array([
[ 12.82 , 8.104 ],
[ 15.499 , 5.425 ],
[ 14.859 , 0.466 ],
[ 12.18 , 3.146 ],
[ 10.0 , -13.75 ],
[ 14.26 , -13.324 ],
[ 14.26 , -15.334 ],
[ 18.24 , -14.936 ],
[ 18.24 , -12.926 ],
[ 22.5 , -12.5 ],
[ 30.0 , 0.0 ],
[ 15.0 , 25.0 ]
]),
np.array([
[ -5.084 , -1.703 ],
[ -5.084 , 0.478 ],
[ -1.416 , -1.117 ],
[ -1.416 , -3.297 ],
[ 2.5 , -5.0 ],
[ 10.0 , -13.75 ],
[ 12.18 , 3.146 ],
[ 14.859 , 0.466 ],
[ 15.499 , 5.425 ],
[ 12.82 , 8.104 ],
[ 15.0 , 25.0 ],
[ -2.5 , 25.0 ],
[ -1.533 , 18.23 ],
[ -3.553 , 18.23 ],
[ -2.987 , 14.27 ],
[ -0.967 , 14.27 ],
[ 0.0 , 7.5 ],
[ -9.0 , -0.0 ]
]),
np.array([
[ -15.279 , 10.465 ],
[ -12.947 , 10.465 ],
[ -10.889 , 7.035 ],
[ -13.221 , 7.035 ],
[ -9.0 , -0.0 ],
[ 0.0 , 7.5 ],
[ -0.967 , 14.27 ],
[ -2.987 , 14.27 ],
[ -3.553 , 18.23 ],
[ -1.533 , 18.23 ],
[ -2.5 , 25.0 ],
[ -15.0 , 25.0 ],
[ -19.5 , 17.5 ]
]),
np.array([
[ -17.218 , -7.609 ],
[ -18.633 , -6.194 ],
[ -15.698 , -3.476 ],
[ -14.282 , -4.891 ],
[ -9.0 , -0.0 ],
[ -13.221 , 7.035 ],
[ -10.889 , 7.035 ],
[ -12.947 , 10.465 ],
[ -15.279 , 10.465 ],
[ -19.5 , 17.5 ],
[ -30.0 , -0.0 ],
[ -22.5 , -12.5 ]
])
])

In [15]:
StudyMultipleMoving(tst)

[ 0.607 -3.618  3.066 -3.618  3.066 -0.035  0.607  2.424 -2.942  2.424
 -5.435  2.424]


interactive(children=(FloatSlider(value=0.0, description='m', step=1.0), IntSlider(value=4000, description='si…

In [9]:
tst = np.array([
np.array([
[ -14.282 , -4.891 ],
[ -15.698 , -3.476 ],
[ -18.633 , -6.194 ],
[ -17.218 , -7.609 ],
[ -22.5 , -12.5 ],
[ -15.0 , -25.0 ],
[ -5.0 , -25.0 ],
[ -1.952 , -16.873 ],
[ 0.184 , -16.873 ],
[ 1.588 , -13.127 ],
[ -0.548 , -13.127 ],
[ 2.5 , -5.0 ],
[ -1.416 , -3.297 ],
[ -1.416 , -1.117 ],
[ -5.084 , 0.478 ],
[ -5.084 , -1.703 ],
[ -9.0 , -0.0 ]
]),
np.array([
[ 18.24 , -12.926 ],
[ 18.24 , -14.936 ],
[ 14.26 , -15.334 ],
[ 14.26 , -13.324 ],
[ 10.0 , -13.75 ],
[ 2.5 , -5.0 ],
[ -0.548 , -13.127 ],
[ 1.588 , -13.127 ],
[ 0.184 , -16.873 ],
[ -1.952 , -16.873 ],
[ -5.0 , -25.0 ],
[ 15.0 , -25.0 ],
[ 22.5 , -12.5 ]
]),
np.array([
[ 12.82 , 8.104 ],
[ 15.499 , 5.425 ],
[ 14.859 , 0.466 ],
[ 12.18 , 3.146 ],
[ 10.0 , -13.75 ],
[ 14.26 , -13.324 ],
[ 14.26 , -15.334 ],
[ 18.24 , -14.936 ],
[ 18.24 , -12.926 ],
[ 22.5 , -12.5 ],
[ 30.0 , 0.0 ],
[ 15.0 , 25.0 ]
]),
np.array([
[ -5.084 , -1.703 ],
[ -5.084 , 0.478 ],
[ -1.416 , -1.117 ],
[ -1.416 , -3.297 ],
[ 2.5 , -5.0 ],
[ 10.0 , -13.75 ],
[ 12.18 , 3.146 ],
[ 14.859 , 0.466 ],
[ 15.499 , 5.425 ],
[ 12.82 , 8.104 ],
[ 15.0 , 25.0 ],
[ -2.5 , 25.0 ],
[ 0.0 , 7.5 ],
[ -9.0 , -0.0 ]
]),
np.array([
[ -19.5 , 17.5 ],
[ -15.279 , 10.465 ],
[ -12.947 , 10.465 ],
[ -10.889 , 7.035 ],
[ -13.221 , 7.035 ],
[ -9.0 , -0.0 ],
[ 0.0 , 7.5 ],
[ -2.5 , 25.0 ],
[ -15.0 , 25.0 ]
]),
np.array([
[ -17.218 , -7.609 ],
[ -18.633 , -6.194 ],
[ -15.698 , -3.476 ],
[ -14.282 , -4.891 ],
[ -9.0 , -0.0 ],
[ -13.221 , 7.035 ],
[ -10.889 , 7.035 ],
[ -12.947 , 10.465 ],
[ -15.279 , 10.465 ],
[ -19.5 , 17.5 ],
[ -30.0 , -0.0 ],
[ -22.5 , -12.5 ]
])
])